In [1]:
import pandas as pd
import requests
import xmltodict

In [63]:
url = 'http://ergast.com/api/f1/2022/10/qualifying'

payload={}
headers={}

response = requests.request("GET", url, headers=headers, data=payload)

dict_data = xmltodict.parse(response.content)
dict_data['MRData']['@total']

'0'

In [8]:
races_total = []
race_year = []
for year in range(2014,2023):
    url = f'http://ergast.com/api/f1/{year}'
    payload={}
    headers={}
    response = requests.request("GET", url, headers=headers, data=payload)
    dict_data = xmltodict.parse(response.content)
    race_year.append(dict_data['MRData']['RaceTable']['@season'])
    races_total.append(dict_data['MRData']['@total'])
total_races_dict = {'year': race_year, 'total': races_total}
total_races_df = pd.DataFrame(total_races_dict)
total_races_df

In [125]:
dict_list = []
print("looping through each year (2014-2021) and each race to grab pole position time")
for i in range(2014,2022):
    race_name = []
    pole_position_times = []
    length = total_races_df[total_races_df['year'] == f'{i}']['total']
    for j in range(1,int(length)+1):
        url = f'http://ergast.com/api/f1/{i}/{j}/qualifying'
        payload={}
        headers={}
        response = requests.request("GET", url, headers=headers, data=payload)
        dict_data = xmltodict.parse(response.content)

        if (dict_data['MRData']['@total'] == '0'):
            break;
        elif 'Q3' in dict_data['MRData']['RaceTable']['Race']['QualifyingList']['QualifyingResult'][0]:
            pole_position_times.append(dict_data['MRData']['RaceTable']['Race']['QualifyingList']['QualifyingResult'][0]['Q3'])
        elif 'Q2' in dict_data['MRData']['RaceTable']['Race']['QualifyingList']['QualifyingResult'][0]:
            pole_position_times.append(dict_data['MRData']['RaceTable']['Race']['QualifyingList']['QualifyingResult'][0]['Q2'])
        else:
            pole_position_times.append(dict_data['MRData']['RaceTable']['Race']['QualifyingList']['QualifyingResult'][0]['Q1'])
        
        race_name.append(dict_data['MRData']['RaceTable']['Race']['RaceName'])

    dict_list.append({'year': i, 'race': race_name, 'pole_position': pole_position_times})
    print(f'year {i} completed')
    # pole_pos_df = pd.DataFrame(pole_pos_dict)
print("all times have been added to dict_list")

looping through each year (2014-2021) and each race to grab pole position time
year 2014 completed
year 2015 completed
year 2016 completed
year 2017 completed
year 2018 completed
year 2019 completed
year 2020 completed
year 2021 completed
all times have been added to dict_list


In [123]:
from datetime import datetime

rows1=[]
rows2=[]
rows3=[]
for data in dict_list:
    race = data['race']
    year = data['year']
    pole = data['pole_position']
    for row in race:
        rows1.append(row)
    for row in pole:
        row = datetime.strptime(row, '%M:%S.%f').time() # convert time str to timestamp
        rows2.append(row)
        rows3.append(year)
    
rows_dict={"year": rows3,"race": rows1, "pole":rows2}
rows_df = pd.DataFrame(rows_dict)
rows_df.style.set_caption('Pole Position Results 2014-2021')

,year,race,pole
0,2014,Australian Grand Prix,00:01:44.231000
1,2014,Malaysian Grand Prix,00:01:59.431000
2,2014,Bahrain Grand Prix,00:01:33.185000
3,2014,Chinese Grand Prix,00:01:53.860000
4,2014,Spanish Grand Prix,00:01:25.232000
5,2014,Monaco Grand Prix,00:01:15.989000
6,2014,Canadian Grand Prix,00:01:14.874000
7,2014,Austrian Grand Prix,00:01:08.759000
8,2014,British Grand Prix,00:01:35.766000
9,2014,German Grand Prix,00:01:16.540000


In [64]:
race_name = []
pole_position_times = []
for i in range(1,10):
    url = f'http://ergast.com/api/f1/2022/{i}/qualifying'
    payload={}
    headers={}

    response = requests.request("GET", url, headers=headers, data=payload)
    dict_data = xmltodict.parse(response.content)
    pole_position_times.append(dict_data['MRData']['RaceTable']['Race']['QualifyingList']['QualifyingResult'][0]['Q3'])
    race_name.append(dict_data['MRData']['RaceTable']['Race']['RaceName'])
    
pole_pos_dict = {'race': race_name, 'pole_position': pole_position_times}
pole_pos_df_22 = pd.DataFrame(pole_pos_dict)

In [65]:
pole_pos_df_22

,race,pole_position
0,Bahrain Grand Prix,1:30.558
1,Saudi Arabian Grand Prix,1:28.200
2,Australian Grand Prix,1:17.868
3,Emilia Romagna Grand Prix,1:27.999
4,Miami Grand Prix,1:28.796
5,Spanish Grand Prix,1:18.750
6,Monaco Grand Prix,1:11.376
7,Azerbaijan Grand Prix,1:41.359
8,Canadian Grand Prix,1:21.299
